In [ ]:
import numpy as np
import os  
from scipy.ndimage import filters  
from PIL import Image  
from pylab import *  
import cv2
from matplotlib import pyplot as plt

def get_gradients(img):
    """ Compute the gradient of x, y, digonal 45 and 
        digonal 135 directions. """
    
    rows, cols = img.shape
    Ix2 = np.zeros((rows, cols))
    Iy2 = np.zeros((rows, cols))
    Ixy = np.zeros((rows, cols))
    Iu45 = np.zeros((rows, cols))
    Iv135 = np.zeros((rows, cols))
    # box filter for x direction
    fx = np.array([[-1, 0, 1],
                   [-1, 0, 1],
                   [-1, 0, 1]])
    # box filter for y direction
    fy = np.array([[-1, -1, -1],
                   [0,  0,  0, ],
                   [1,  1,  1]])
    # calculate the gradient of four directions for each pixel
    for r in range(1, rows-1):
        for c in range(1, cols-1):
            p_m = img[r-1:r+2, c-1:c+2]
            Ix = (p_m * fx).sum()
            Iy = (p_m * fy).sum()
            Ix2[r,c] = Ix * Ix
            Iy2[r,c] = Iy * Iy
            Ixy[r,c] = Ix * Iy
            Iu45[r,c] = pow(img.item(r+1, c-1) - img.item(r-1, c+1), 2)
            Iv135[r,c] = pow(img.item(r-1, c-1) - img.item(r+1, c+1), 2)
    return Ix2, Iy2, Ixy, Iu45, Iv135

def gauss_filter(img, delta=3):
    """ A simiple gaussian filter but slooow"""
    
    rows, cols = img.shape
    filter_out = np.zeros((rows,cols), dtype = float)
    # the distance map
    kernel = np.array([[2., 1., 2.],
                       [1., 0., 1.],
                       [2., 1., 2.]])
    # gaussican kernel
    kernel = np.exp(-kernel / (2*(delta**2)))
    # nomalization
    kernel = kernel / kernel.sum()
    
    # Not consider about the border!
    for r in range(1, rows-1):
        for c in range(1, cols-1):
            filter_out[r,c] = (img[(r-1):(r+2), (c-1):(c+2)] * kernel).sum()

    return filter_out

def detect_harris_Kpoint(Ix2, Iy2, Ixy, K=4):
    """ Dectect K*K harris points, each for one block."""
    
    key_pts = np.zeros((K**2,3), dtype=int)
    # get the filtered gradients
    Ix2 = gauss_filter(Ix2)
    Iy2 = gauss_filter(Iy2)
    Ixy2 = gauss_filter(Ixy*Ixy)
    
    # obtain the harris response
    R = (Ix2*Iy2 - Ixy2) / (Ix2 + Iy2 + 0.0001)
    
    # set the threshold, but it is not useful here
    thresh_hold = R.max()*0.05
    R[R < thresh_hold] = 0
    rows, cols = R.shape
    # block size
    blockr = int(rows / K)
    blockc = int(cols / K)

    # get one key point with strongest response for each block
    for i in range(K):
        for j in range(K):
            for r in range(blockr):
                for c in range(blockc):
                    pixel_r = i*blockr + r
                    pixel_c = j*blockc + c
                    pt_index = i*K +j
                    if R[pixel_r,pixel_c] > key_pts[pt_index,2]:
                        key_pts[pt_index,0] = pixel_r
                        key_pts[pt_index,1] = pixel_c
                        key_pts[pt_index,2] = R[pixel_r,pixel_c]
    return key_pts, R

def compute_harris_response(im, sigma=3):  
    """ Compute the Harris corner detector response function  
        for each pixel in a graylevel image. """  
  
    # derivatives  
    imx = np.zeros(im.shape)  
    filters.gaussian_filter(im, (sigma, sigma), (0, 1), imx)  
    imy = np.zeros(im.shape)  
    filters.gaussian_filter(im, (sigma, sigma), (1, 0), imy)  
  
    # compute components of the Harris matrix  
    Wxx = filters.gaussian_filter(imx * imx, sigma)  
    Wxy = filters.gaussian_filter(imx * imy, sigma)  
    Wyy = filters.gaussian_filter(imy * imy, sigma)  
  
    # determinant and trace  
    Wdet = Wxx * Wyy - Wxy ** 2  
    Wtr = Wxx + Wyy + 0.00001
  
    return Wdet / Wtr

def get_harris_points(harrisim, min_dist=80, threshold=0.05):  
    """ Return corners from a Harris response image  
        min_dist is the minimum number of pixels separating  
        corners and image boundary. """  
  
    # find top corner candidates above a threshold  
    corner_threshold = harrisim.max() * threshold  
    harrisim_t = (harrisim > corner_threshold) * 1  
  
    # get coordinates of candidates  
    coords = np.array(harrisim_t.nonzero()).T  
  
    # ...and their values  
    candidate_values = [harrisim[c[0], c[1]] for c in coords]  
  
    # sort candidates (reverse to get descending order)  
    index = argsort(candidate_values)[::-1]  
  
    # store allowed point locations in array  
    allowed_locations = np.zeros(harrisim.shape)  
    allowed_locations[min_dist:-min_dist, min_dist:-min_dist] = 1  
  
    # select the best points taking min_distance into account  
    filtered_coords = []
    for i in index:  
        if allowed_locations[coords[i, 0], coords[i, 1]] == 1:  
            filtered_coords.append(coords[i])  
            allowed_locations[(coords[i, 0] - min_dist):(coords[i, 0] + min_dist),  
            (coords[i, 1] - min_dist):(coords[i, 1] + min_dist)] = 0  
  
    return filtered_coords  

def get_acculmulated_histgram(in_img):
    """ Caculate the histogram of four gradient maps of four directions."""
    
    rows, cols = in_img.shape
    hori_histogram = np.zeros(cols, dtype = int)
    ver_histogram = np.zeros(rows, dtype = int)
    dia_45_histogram = np.zeros(rows + cols - 1, dtype = int)
    length45 = np.zeros(rows + cols - 1, dtype = int)
    dia_135_histogram = np.zeros(rows + cols - 1, dtype = int)
    length135 = np.zeros(rows + cols - 1, dtype = int)
    Ix2, Iy2, Ixy, Iu45, Iv135 = get_gradients(in_img)
    
    # get the harris points using the gradient map
    key_points, R = detect_harris_Kpoint(Ix2, Iy2, Ixy)
    
    # fill the histogram of the gradient maps
    for r in range(rows):
        for c in range(cols):
            hori_histogram[c] += Ix2[r,c] 
            ver_histogram[r] += Iy2[r,c]
            index135 = r + c
            index45 = rows - r + c - 1
            dia_45_histogram[index45] += Iu45[r,c]
            length45[index45] += 1
            dia_135_histogram[index135] += Iv135[r,c]
            length135[index135] += 1
    
    # normalize the histogram with the pixel number of each bin
    hori_histogram = hori_histogram / rows
    ver_histogram = ver_histogram / cols
    dia_45_histogram = dia_45_histogram / length45
    dia_135_histogram = dia_135_histogram / length135
    
#     plt.subplot(221),plt.plot([i for i in range(cols)], hori_histogram),plt.title('hori_')
#     plt.subplot(222),plt.plot([i for i in range(rows)], ver_histogram),plt.title('ver_')
#     plt.subplot(223),plt.plot([i for i in range(rows + cols - 1)], dia_45_histogram),plt.title('dia_45_')
#     plt.subplot(224),plt.plot([i for i in range(rows + cols - 1)], dia_135_histogram),plt.title('dia_135_')
#     plt.show()
    
#     plt.imshow(in_img, cmap=plt.cm.gray_r)
#     plt.scatter(key_points[:,1], key_points[:,0], color='red')
#     plt.show()
    
#     plt.imshow(Ix2, cmap=plt.cm.gray_r)
#     plt.show()
#     plt.imshow(Iy2, cmap=plt.cm.gray_r)
#     plt.show()
#     plt.imshow(Iu45, cmap=plt.cm.gray_r)
#     plt.show();
#     plt.imshow(Iv135, cmap=plt.cm.gray_r)
#     plt.show();
    return hori_histogram, ver_histogram, dia_45_histogram, dia_135_histogram, key_points, R

def get_xy_trans(histo_p1, histo_p2, maxoffset=40):
    """ Compute the transformation using the histogram."""
    
    translation = 0
    # set a max number
    max_conv = 1e10000
    
    # the offset that make (x-y)^2 minimum is the right transformation
    for offset in range(-maxoffset, maxoffset+1):
        conv = ((histo_p2[maxoffset:-1-maxoffset] - histo_p1[maxoffset + offset:-1-maxoffset + offset])**2).sum()
        if conv < max_conv:
            max_conv = conv
            translation = offset
    return translation

def get_correspondence_pt(pre_img, pre_keys, rough_tx, rough_ty, in_img, in_keys):
    """ Fing the point pair using the transformation roughly caculated by the convolution
        of the histogram."""
    
    pre_rows,pre_cols = pre_keys.shape
    in_rows, in_cols = in_keys.shape
    # the maximum number of point pairs
    if pre_rows >= in_rows:
        max_rows = pre_rows
    else:
        max_rows = in_rows
        
    pt_pairs = np.zeros((max_rows,4), dtype=int)
    
    # for each key point in pre image, find the nearest key point in the coming image, considering the rough transformation
    pt_index = 0
    for k in range(pre_rows):
        min_dist = 10**100
        for kk in range(in_rows):
            dist = pow(pre_keys[k,0] - in_keys[kk,0] - rough_ty,2.0) + pow(pre_keys[k,1] - in_keys[kk,1] - rough_tx,2.0)
            # the point pair should have the smallest distance and the distance should Not bigger than 5 pixel in each direction
            if (dist < min_dist and dist < 50):
                pt_pairs[pt_index, 0] = pre_keys[k,0]
                pt_pairs[pt_index, 1] = pre_keys[k,1]
                pt_pairs[pt_index, 2] = in_keys[kk,0] + rough_ty
                pt_pairs[pt_index, 3] = in_keys[kk,1] + rough_tx
                min_dist = dist
                pt_index += 1
                
    print("the points pairs number: ", pt_index)
    return pt_pairs[0:pt_index,:]

def my_ransac_xy(all_pt_pairs):
    """ Adjust the rough tx and ty using the random sample consensus method """
    
    pt_nums,cols = all_pt_pairs.shape
    # intial the adjusted transformation
    adjust_tx = 0.
    adjust_ty = 0.
    batch_num = 8
    # ... and if there are not enough numbers of point pairs return the 
    # initial number
    if pt_nums < batch_num:
        print("error: not enough point pairs")
        return adjust_tx, adjust_ty
    
    picked_pt = np.zeros((batch_num,cols), dtype=float)
    selected_ones = []
    # the maximum number of the iteration
    max_iteration = 50
    
    iteration = 0
    max_inliners = 0
    final_tx = 0.
    final_ty = 0.
    
    while 1:
        pick_num = 0
        while (pick_num < batch_num):
            picked_one = np.random.randint(0,pt_nums-1)
            # all the point pairs in the same batch should be different
            if (picked_one in selected_ones):
                continue
            selected_ones.append(picked_one)
            pick_num += 1
        # save the picked ones
        picked_pt[:,:] = all_pt_pairs[selected_ones,:]
        selected_ones = []

        # adjusted value is the average of the transformation of all the point pairs
        adjust_tx = ((picked_pt[:,1] - picked_pt[:,3]).sum(axis=0) / batch_num)
        adjust_ty = ((picked_pt[:,0] - picked_pt[:,2]).sum(axis=0) / batch_num)
#         print("adjsted = ",adjust_tx,adjust_ty)
        inliners = 0
        for k in range(pt_nums):
            newr = all_pt_pairs[k,2] + adjust_ty
            newc = all_pt_pairs[k,3] + adjust_tx
            # count the inner points
            if ((pow(newr - all_pt_pairs[k,0],2) + pow(newc - all_pt_pairs[k,1],2)) < 3):
                inliners += 1
                
#         print("inliners = ", inliners)
        # if the amount of the inner points is big eough, return directly
        if (inliners > pt_nums * 0.9):
            return adjust_tx, adjust_ty
        # or we will choose the ones with the most inners
        if (inliners > max_inliners):
            max_inliners = inliners
            print("max_inliners = ", max_inliners)
            final_tx = adjust_tx
            final_ty = adjust_ty
        
        iteration += 1
        # if the maximum iteration is reached, return the best ones found so far
        if ((iteration > max_iteration)):
            return final_tx, final_ty

def get_H(pt_pairs):
    """ Caculate the homography matrix using eigenvalue decomposition methond.
        Input is point pairs, the number should be greater than 4."""
    
    rows, cols = pt_pairs.shape
    # the format of the point pair should be x1,y1,x2,y2 and the number should be at least 4.
    if (cols != 4 or rows < 4):
        print("the points pairs size is not right!")
        return
    # rewrite the projective transformation in the form of Ah = 0
    # and fullfil the A matrix with each point pair
    A = np.zeros((2*rows,9), dtype = float)
    for k in range(rows):
        A[2*k,:] = np.array([pt_pairs[k,2], pt_pairs[k,3], 1., 0.,0.,0., 
                            -pt_pairs[k,0]* pt_pairs[k,2],
                            -pt_pairs[k,0]* pt_pairs[k,3],
                            -pt_pairs[k,0]])
        A[2*k+1,:] = np.array([0.,0.,0.,
                               pt_pairs[k,2], 
                               pt_pairs[k,3], 1., 
                              -pt_pairs[k,1]*pt_pairs[k,2],
                              -pt_pairs[k,1]*pt_pairs[k,3],
                              -pt_pairs[k,1]])
    A = A / np.max(abs(A))
    # eigenvalue decompositon
    w,b = np.linalg.eig(np.dot(A.transpose(),A))
    # fing the index of the minmum eigenvalue
    min_index = np.where(w == np.min(w))
    # and the eigenvector of the minmum eigenvalue is the H
    H = b[:,min_index].reshape(3,3)
    # set the last element of the H to 1
    H = H / H[2,2]
    return H

def colinear(pt1,pt2,pt3):
    """ Estimate the three points are colinear or not. """
    
    vct1 = pt2 - pt1
    vct2 = pt3 - pt1
    cos_theta = ((vct1 * vct2).sum()) / (sqrt(np.dot(vct1,vct1.transpose()))*sqrt(np.dot(vct2,vct2.transpose())))
    # if the angle of the two vector is near 0 or 180, we think they are colinear.
    if (abs(cos_theta)) < 0.95:
        return False
    else:
        return True

def my_ransac_H(all_pt_pairs, thresh_hold = 2):
    """ Using random sample consensus method to find the H with the most inners.
        Input: the point pairs in the form of x1,y1,x2,y2
        Output: H and inner point pairs"""
    
    pt_nums,cols = all_pt_pairs.shape
    # each time use 4 point-pairs to caculate a H
    batch_nums = 4
    H_final = np.array([[1.,0.,0.],
                        [0.,1.,0.],
                        [0.,0.,1.]])
    in_liners = np.zeros((pt_nums,4), dtype=float)
    
    if pt_nums < batch_nums:
        print("error: not enough point pairs")
        return H_final, in_liners
    
    picked_pt = np.zeros((batch_nums,cols), dtype=float)
    # the maximum number of iteration for the caculation of H
    max_iteration = 100
    # the maximum number of iteration to pick the four point-pairs
    max_iteration_pickpt = 50
    
    iteration = 0
    max_inliners = 0

    while 1:
        pick_num = 0
        iteration_pick = 0
        selected_ones = []
        while (pick_num < batch_nums):
            iteration_pick += 1
            # can not find enough non-colinear points, return the best one being found so far
            if (iteration_pick > max_iteration_pickpt):
                return H_final,in_liners
            picked_one = np.random.randint(0,pt_nums-1)
            # all the points should be different
            if (picked_one in selected_ones):
                continue
            # every three of them should Not be colinear
            if (pick_num == 2):
                p1 = all_pt_pairs[selected_ones[0],0:2]
                p2 = all_pt_pairs[selected_ones[1],0:2]
                p3 = all_pt_pairs[picked_one,0:2]
                if (colinear(p1,p2,p3)):
                    continue
            if (pick_num == 3):
                p1 = all_pt_pairs[selected_ones[0],0:2]
                p2 = all_pt_pairs[selected_ones[1],0:2]
                p3 = all_pt_pairs[selected_ones[2],0:2]
                p4 = all_pt_pairs[picked_one,0:2]
                if((colinear(p1,p2,p4)) or (colinear(p1,p3,p4)) or (colinear(p2,p3,p4))):
                    continue
            selected_ones.append(picked_one)
            pick_num += 1
        # set the picked ones
        picked_pt[:,:] = all_pt_pairs[selected_ones,:]
        selected_ones = []
        # ... and use them to caculate the H
        H_temp = get_H(picked_pt)
        
        inliners = 0
        selected_ones = []
        for k in range(pt_nums):
            newxy = np.dot(H_temp, [all_pt_pairs[k,2],all_pt_pairs[k,3],1.])
            newxy = newxy / newxy[2]
            # pick out all the inners
            if ((pow(newxy[0] - all_pt_pairs[k,0],2) + pow(newxy[1] - all_pt_pairs[k,1],2)) < thresh_hold):
                selected_ones.append(k)
                inliners += 1
        # update the inner set
        in_subset = all_pt_pairs[selected_ones,:]
        print("inliners = ", inliners)
        # if the amount of the inners is big enough, return directly
        if (inliners > pt_nums*0.9):
            return H_temp, in_subset
        # store the best H and its inners
        if (inliners > max_inliners):
            max_inliners = inliners
            print("max_inliners = ", max_inliners)
            H_final = H_temp
            in_liners[0:inliners,:] = in_subset
        # if the inners are good enough, update the iteration number or we think this iteration is false
        if inliners > batch_nums/2:
            iteration += 1
        # the iteration is over, return the best H having being found so far and its inners
        if ((iteration > max_iteration)):
            return H_final, in_liners
        
def seam_cut(pre_img, in_img, tx, ty):
    """ Caculate the best seam line. Not used now."""
    
    if (tx < 0):
        print("tx < 0, out!")
        return
    rows, cols = in_img.shape
    cut_line = np.zeros((rows,2), dtype= int)
    
    if (ty < 0):
        overlap_in = pre_img[0:rows+ty,tx::]
        overlap_dst = in_img[0-ty::,0:cols-tx]
    if (ty >= 0):
        overlap_in = pre_img[ty::,tx::]
        overlap_dst = in_img[0:rows-ty,0:cols-tx]
    rows, cols = overlap_in.shape
    print(overlap_in.shape)
    print(overlap_dst.shape)
    
    overlap_in.astype(float)
    overlap_dst.astype(float)
    print(overlap_in.dtype)
    dis = ((overlap_in - overlap_dst)**2.)
    print(dis.dtype)
    
    for i in range(1,rows):
        for j in range(cols):
            if (j == 0):
                min_neibor = dis[i-1,j]
                if (dis[i-1, j+1] <= min_neibor):
                    min_neibor = dis[i-1,j+1]
                dis[i,j] = dis[i,j] + min_neibor
                continue
                
            if (j == cols-1):
                min_neibor = dis[i-1,j]
                if (dis[i-1, j-1] <= min_neibor):
                    min_neibor = dis[i-1,j-1]
                dis[i,j] = dis[i,j] + min_neibor
                continue
                
            min_neibor = dis[i-1,j]
            if (dis[i-1,j+1] < min_neibor):
                min_neibor = dis[i-1,j+1]
            if (dis[i-1,j-1] < min_neibor):
                min_neibor = dis[i-1,j-1]
#             if (dis[i-1,j+1] < min_neibor):
#                 min_neibor = dis[i-1,j+1]
            dis[i,j] = dis[i,j] + min_neibor
    
    min_err = dis[int(rows/2), int(cols/2)]
    min_c = 0
    for c in range(cols-2,1,-1):
        if (dis[rows-1,c] < min_err):
            min_err = dis[rows-1,c]
            min_c = c
    if(ty > 0):
        cut_r = rows-1
    else:
        cut_r = rows-1-ty
    cut_line[cut_r, 0] = rows-1
    cut_line[cut_r, 1] = min_c
    
    for r in range(rows-2,-1,-1):
        min_mid = min_c
        
        if(min_c == 0):
            min_err = dis[r, min_c]
            if (dis[r, min_c+1] <= min_err):
                min_mid = min_c +1
            min_c = min_mid
            
            if(ty > 0):
                cut_r = r
            else:
                cut_r = r-ty
            cut_line[cut_r, 0] = cut_r
            cut_line[cut_r, 1] = min_c
            continue
            
        if(min_c == cols-1):
            min_err = dis[r, min_c]
            if (dis[r, min_c-1] <= min_err):
                min_mid = min_c -1
            min_c = min_mid
            
            if(ty > 0):
                cut_r = r
            else:
                cut_r = r-ty
            cut_line[cut_r, 0] = cut_r
            cut_line[cut_r, 1] = min_c
            continue
        
        min_err = dis[r, min_c]
        if (dis[r, min_c-1] < min_err):
            min_mid = min_c-1
            min_err = dis[r, min_c-1]
        if (dis[r, min_c+1] < min_err):
            min_mid = min_c+1
            min_err = dis[r, min_c+1]
        min_c = min_mid
        if(ty > 0):
            cut_r = r
        else:
            cut_r = r-ty
        cut_line[cut_r, 0] = cut_r
        cut_line[cut_r, 1] = min_c

    return cut_line

def stitch_to_overlap(final_pano, next_img, Ht):
    """ Project the coming image to the final panorama image using the caculated homography
        matrix and directly replace the pixel in the panorama."""
    
    rows_in,cols_in = next_img.shape
    rows_dst,cols_dst = final_pano.shape
    for r in range(rows_in):
        for c in range(cols_in):
            # caculate the final coordinate of each pixel in the current image
            newxy = np.dot(Ht, [r,c,1])
            newxy = newxy / newxy[2]
            if ((newxy[0] < 0) or (newxy[0] > rows_dst-1) or (newxy[1] < 0) or (newxy[1] > cols_dst-1)):
                continue
            # set the pixel value in the panorama image
            final_pano[int(newxy[0]), int(newxy[1])] = next_img[r,c]
    return final_pano

In [ ]:
# read the video
cap = cv2.VideoCapture("E:\\alls\\VID_20100115_195346.3gp")
# get the total number of the frames
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("the length of the video is :", length)
# get one frame
ret, first_frame = cap.read()
# ... and turn it into gray style
gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
# ... and rotate it into the regular angle
gray = cv2.flip(cv2.flip(np.rot90(gray),0),1)

# initialize the panorama, its width is N times the width of the video
final_pano = np.zeros((gray.shape[0], gray.shape[1] * 3), dtype=int)

# obtain the harris points
harris_res = compute_harris_response(gray)
all_harris = get_harris_points(harris_res)
keys = np.array(all_harris)
print(keys.shape)
# ... and the histogram of four directions
HH1, VH1, DH45, DH135, my_keypts, R = get_acculmulated_histgram(gray)
H_total = np.array([[1.,0.,0.],
                    [0.,1.,0.],
                    [0.,0.,1.]])
for i in range(int(length/2)):
    print("the iteration: ", i)
    # do one caculation every two frames
    for s in range(2):
        ret1, frame1 = cap.read()
    
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.flip(cv2.flip(np.rot90(gray1),0),1)

    harris_res = compute_harris_response(gray1)
    all_harris = get_harris_points(harris_res)
    keys2 = np.array(all_harris)    
    print(keys2.shape)
    
    HH2, VH2, DH452, DH1352, knone, Rnone = get_acculmulated_histgram(gray1)

    tx = get_xy_trans(HH1, HH2)
    # we just allow the camera to move forword for now
    if tx < 0:
        continue
    ty = get_xy_trans(VH1, VH2)
    tu = get_xy_trans(DH45, DH452)
    tv = get_xy_trans(DH135, DH1352)
    # adjust the x, y transformation using the diagonal ones
    tx = (tx + 1.414/2.0 * tu + 1.414/2. * tv) / 2.
    ty = (ty - 1.414/2.0 * tu + 1.414/2. * tv) / 2.
    print("tx =",tx)
    print("ty =",ty)
    
    # obtain the point pairs
    points = get_correspondence_pt(gray, keys, tx, ty, gray1, keys2)
    # ... and adjust the values with these points
    adjusted_tx, adjusted_ty = my_ransac_xy(points)
    print("adjusted_tx =",adjusted_tx)
    print("adjusted_ty =",adjusted_ty)
    tx = tx + adjusted_tx
    ty = ty + adjusted_ty
    print("tx =",tx)
    print("ty =",ty)
    # update the coordinate of the new point pairs
    points[:,2] = points[:,2] + adjusted_ty
    points[:,3] = points[:,3] + adjusted_tx
    
    # caculate the error using tx, ty only
    error_xy = (pow(points[:,0]-points[:,2],2)+pow(points[:,1]-points[:,3],2)).sum()
    print("error after tx,ty:",error_xy)
    
    # obtain the homography matrix 
    H_new, subset = my_ransac_H(points,3)
    # ... and improve the result using the inners and a low threshold
    H_new, subset = my_ransac_H(subset,1.5)
    
    # plot the point set 
    plt.scatter(points[:,1], points[:,0], color='red')
    point_trans = np.zeros((3,points.shape[0]))
    point_trans[0,:] = points[:,2]
    point_trans[1,:] = points[:,3]
    point_trans[2,:] = 1
    point_trans = np.dot(H_new, point_trans)
    point_trans[0,:] = point_trans[0,:] / point_trans[2,:]
    point_trans[1,:] = point_trans[1,:] / point_trans[2,:]
    plt.scatter(point_trans[1,:], point_trans[0,:], marker = 'x', color='blue')
    plt.show()
    
    # caculate the error using tx,ty and H
    error_xyH = (pow(points[:,0]-point_trans[0,:],2)+pow(points[:,1]-point_trans[1,:],2)).sum()
    print("error after tx,ty,H:",error_xyH)
    
    # it should be lower, but if the error increase for some reason, we dismiss it
    if error_xyH > error_xy:
        H_new = np.array([[1.,0.,0.],
                          [0.,1.,0.],
                          [0.,0.,1.]])
    # multiply tx, ty, and H, obtain the total transformation for the current frame
    transH = np.array([[1., 0., ty],
                       [0., 1., tx],
                       [0., 0., 1.]])
    H_new = np.dot(H_new, transH)
    H_new = H_new / H_new[2,2]
    # ... and accumulated with the previous one
    H_total = np.dot(H_total, H_new)
    H_total = H_total / H_total[2,2]
    # stitch to the panorama
    final_pano = stitch_to_overlap(final_pano, gray1, H_total)

    HH1 = HH2
    VH1 = VH2
    DH45 = DH452
    DH135 = DH1352
    gray = gray1
    keys = keys2
    # save the panorame each time for debugging only
    cv2.imwrite("final_pano-3.png", final_pano)
    
#cv2.imwrite("final_pano-3.png", final_pano)
plt.imshow(final_pano, cmap=plt.cm.gray_r)
plt.show()

In [ ]:
# print((pow(pt_inliner[:,1] - pt_inliner[:,3], 2) + pow(pt_inliner[:,0] - pt_inliner[:,2], 2)).sum(axis=0))
# pt_transed = (np.dot(R, pt_inliner[:,2:4].transpose()) + T).transpose()
# print((pow(pt_inliner[:,1] - pt_transed[:,1], 2) + pow(pt_inliner[:,0] - pt_transed[:,0], 2)).sum(axis=0))

# plt.scatter(keys[:,1], keys[:,0], color='red', marker='o')
# plt.scatter(keys2[:,1], keys2[:,0], color='blue', marker='x')
# plt.show()
# plt.scatter(pt_inliner[:,1], pt_inliner[:,0], color='red', marker='o')
# plt.scatter(pt_inliner[:,3], pt_inliner[:,2], color='blue', marker='x')
# plt.show()
# plt.scatter(pt_inliner[:,1], pt_inliner[:,0], color='red', marker='o')
# plt.scatter(pt_transed[:,1], pt_transed[:,0], color='blue', marker='x')
# plt.show()

In [ ]:
# plt.imshow((gray+gray1)/2, cmap=plt.cm.gray)
# plt.show();
# rows, cols = gray1.shape
# Ttotal = T + np.dot(R, [[ty],[tx]])
# gray_TR = np.zeros((rows,cols), dtype=int)
# print(Ttotal)
# for r in range(rows):
#     for c in range(cols):
#         newrc = np.dot(R, [[r],[c]]) + Ttotal
#         newrc = np.round(newrc)
#         if ((newrc[0] > 0) and (newrc[0] < rows) and (newrc[1] > 0) and (newrc[1] < cols)):
#             gray_TR[int(newrc[0]), int(newrc[1])] = gray1[r,c]            

# print(rows,cols)
# print(newrc)
# plt.imshow((gray+gray_TR)/2, cmap=plt.cm.gray)
# plt.show();